# Dense Layer

각 뉴런들은 서로 다른 weight와 bias를 가진 parametric function으로, filtering 연산으로 볼 수 있다.
따라서 여러개의 뉴런들을 묶은 하나의 층(Dense Layer)는 서로 다른 parametric funciton의 묶음

Filter bank의 개념 : 같은 값을 넣더라도 다른값이 나온다.
    
Deep Learning : cascaded(layer가 쌓아짐) filter bank 구조

보통 Dense Layer는 모든 노드가 연결된 fully connected layer를 말한다.

network는 edge(connection)와 node로 구성된다.

sigmoid 함수는 파라미터가 없지만, activation layer라고 부르기도 한다.

각 Layer의 출력값은 다음 layer의 input 역할을 한다.

## shape

**Pytorch 기준**

X = (샘플수, 변수개수) 일때,

Weight = (뉴런개수, 변수개수)

Bias = (뉴런개수)

Output = (샘플수, 뉴런개수)

## 실습

### shapes of dense layers

In [14]:
import torch

N, n_feature = 8, 10
X = torch.rand(N, n_feature)

n_neuron = 3
Linear = torch.nn.Linear(in_features = n_feature, out_features = n_neuron)
Sigmoid = torch.nn.Sigmoid()

Y = Sigmoid(Linear(X))

W = Linear.weight
B = Linear.bias


print(f'X: {X.detach().numpy().shape}')
print(f'Weight: {W.detach().numpy().shape}')
print(f'Bias: {B.detach().numpy().shape}')
print(f'y : {Y.detach().numpy().shape}')

X: (8, 10)
Weight: (3, 10)
Bias: (3,)
y : (8, 3)


### shapes of cascaded dense layers

In [15]:
import torch

N, n_feature = 4, 10
X = torch.rand(N, n_feature)

n_neurons = [3, 5]
Linear1 = torch.nn.Linear(in_features = n_feature, out_features = n_neurons[0])
Linear2 = torch.nn.Linear(in_features = n_neurons[0], out_features = n_neurons[1])
Sigmoid = torch.nn.Sigmoid()

# forward propagation
A1 = Sigmoid(Linear1(X))
Y = Sigmoid(Linear2(A1))

# get weight/bias
W1 = Linear1.weight
B1 = Linear1.bias

W2 = Linear2.weight
B2 = Linear2.bias

print(f'X: {X.detach().numpy().shape} \n')

print(f'Weight1: {W1.detach().numpy().shape}')
print(f'Bias1: {B1.detach().numpy().shape}')
print(f'A1: {A1.detach().numpy().shape} \n')

print(f'Weight2 : {W2.detach().numpy().shape}')
print(f'Bias2 : {B2.detach().numpy().shape}')
print(f'Y : {Y.detach().numpy().shape}')

X: (4, 10) 

Weight1: (3, 10)
Bias1: (3,)
A1: (4, 3) 

Weight2 : (5, 3)
Bias2 : (5,)
Y : (4, 5)


### dense layers with python list

In [17]:
import torch

N, n_feature = 4, 10
X = torch.rand(N, n_feature)

n_neurons = [n_feature, 10, 20, 30 ,40, 50 ,60 ,70 ,80 ,90 ,100]

ReLU = torch.nn.ReLU()

dense_layers = []
for i in range(0, len(n_neurons)-1):
    Linear = torch.nn.Linear(in_features = n_neurons[i], out_features = n_neurons[i+1])
    dense_layers.append(Linear)

print(f'Input : {X.detach().numpy().shape}')
for dense_idx, dense in enumerate(dense_layers):
    X = dense(X)
    X = ReLU(X)
    print("After dense layer ", dense_idx)
    print(X.detach().numpy().shape, '\n')

Y = X

Input : (4, 10)
After dense layer  0
(4, 10) 

After dense layer  1
(4, 20) 

After dense layer  2
(4, 30) 

After dense layer  3
(4, 40) 

After dense layer  4
(4, 50) 

After dense layer  5
(4, 60) 

After dense layer  6
(4, 70) 

After dense layer  7
(4, 80) 

After dense layer  8
(4, 90) 

After dense layer  9
(4, 100) 



### Model Implementation with Sequential Method

In [23]:
import torch

n_neurons = [3, 4, 5, 6]

Sigmoid = torch.nn.Sigmoid()

model = []
for i in range(0, len(n_neurons)-1):
    model.append(torch.nn.Linear(n_neurons[i], n_neurons[i+1]))
    model.append(Sigmoid)

model = torch.nn.Sequential(*model)
model

Sequential(
  (0): Linear(in_features=3, out_features=4, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=4, out_features=5, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=5, out_features=6, bias=True)
  (5): Sigmoid()
)

### Model implementation with Model-subclassing

In [31]:
import torch

class TestModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.dense1 = torch.nn.Linear(n_feature, 10)
        self.dense2 = torch.nn.Linear(10, 20)
        
        self.ReLU = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.dense1(x)
        x = self.ReLU(x)
        x = self.dense2(x)
        x = self.ReLU(x)
        return x
    
model = TestModel()
model

TestModel(
  (dense1): Linear(in_features=10, out_features=10, bias=True)
  (dense2): Linear(in_features=10, out_features=20, bias=True)
  (ReLU): ReLU()
)

### forward propagation of models

In [30]:
import torch

N, n_feature = 4, 10
X = torch.rand(N, n_feature)

class TestModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.dense1 = torch.nn.Linear(n_feature, 10)
        self.dense2 = torch.nn.Linear(10, 20)
        
        self.ReLU = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.ReLU(self.dense1(x))
        x = self.ReLU(self.dense2(x))
        return x
    
model = TestModel()
Y = model(X)
print(Y)

tensor([[0.1317, 0.2153, 0.4418, 0.0184, 0.0000, 0.0979, 0.0952, 0.1561, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0669, 0.0000, 0.0089, 0.1980,
         0.0998, 0.2156],
        [0.2318, 0.2630, 0.4788, 0.0390, 0.0000, 0.1292, 0.0000, 0.0487, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2188, 0.0000, 0.0552, 0.2272,
         0.2534, 0.3283],
        [0.1088, 0.2664, 0.4972, 0.0335, 0.0000, 0.1365, 0.0776, 0.1855, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0495, 0.0000, 0.0000, 0.1238,
         0.1446, 0.2640],
        [0.2265, 0.3517, 0.6527, 0.0000, 0.0000, 0.1875, 0.0295, 0.0852, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0091, 0.0000, 0.2791, 0.0000, 0.0291, 0.1006,
         0.4052, 0.5522]], grad_fn=<ReluBackward0>)


### Layers in Models

In [35]:
model

Sequential(
  (0): Linear(in_features=3, out_features=4, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=4, out_features=5, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=5, out_features=6, bias=True)
  (5): Sigmoid()
)

In [37]:
import torch

n_neurons = [3, 4, 5, 6]

Sigmoid = torch.nn.Sigmoid()

model = []
for i in range(0, len(n_neurons)-1):
    model.append(torch.nn.Linear(n_neurons[i], n_neurons[i+1]))
    model.append(Sigmoid)

model = torch.nn.Sequential(*model)

for i in range(0, len(model), 2):
    W = model[i].weight
    B = model[i].bias
    
    print(W.detach().numpy().shape, B.detach().numpy().shape)

(4, 3) (4,)
(5, 4) (5,)
(6, 5) (6,)


### Trainable Variables in Models

In [40]:
import torch

n_neurons = [3, 4, 5, 6]

Sigmoid = torch.nn.Sigmoid()

model = []
for i in range(0, len(n_neurons)-1):
    model.append(torch.nn.Linear(n_neurons[i], n_neurons[i+1]))
    model.append(Sigmoid)

model = torch.nn.Sequential(*model)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

for i in range(0, len(model), 2):
    print(count_parameters(model[i]))

16
25
36
